In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import explode, split, regexp_replace
from pyspark.sql.types import StructType, StructField, StringType
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
import matplotlib.pyplot as plt
import re

In [ ]:
# create a SparkSession
spark = SparkSession.builder.appName("TwitterStreaming").getOrCreate()

# create a StreamingContext with batch interval of 1 second
ssc = StreamingContext(spark.sparkContext, 1)
sqlContext = SQLContext(spark)


def preprocessing(rdd):
    words = rdd.map(lambda x: x.split("t_end")).flatMap(lambda x: x).map(lambda x: x.strip())
    words = words.filter(lambda x: len(x) > 0)
    words = words.map(lambda x: re.sub(r'http\S+', '', x))
    words = words.map(lambda x: re.sub(r'@\w+', '', x))
    words = words.map(lambda x: re.sub(r'#', '', x))
    words = words.map(lambda x: re.sub(r'RT', '', x))
    words = words.map(lambda x: re.sub(r':', '', x))
    return words

# create a DStream that reads data from port 5554
lines = ssc.socketTextStream("localhost", 5554)

# preprocess non-empty RDDs and store clean tweets in a Spark DataFrame
clean_tweets = lines.transform(lambda rdd: preprocessing(rdd))
clean_tweets.pprint()

In [ ]:
ssc.start()

In [ ]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
clean_tweets.createOrReplaceTempView("tweets")
top_10_tweets = sqlContext.sql( '"SELECT word FROM tweets"' )
top_10_df = top_10_tweets.toPandas()
new_count += 1
print(top_10_df)

In [ ]:
# stop the SparkSession
ssc.stop()